Get emails and labels + split

In [1]:
import glob
import os
corpus_path = 'C:/Users/Rik/Documents/corpus/'
emails, labels = [], []
def readFilesForFolder(folder_path, is_spam):
    file_path = corpus_path + 'enron1/' + folder_path
    for filename in glob.glob(os.path.join(file_path, '*.txt')):
        with open(filename, 'r', encoding="ISO-8859-1") as infile:
            emails.append(infile.read())
            labels.append(is_spam)
readFilesForFolder('spam', 1)
print(len(emails))
print(len(labels))
readFilesForFolder('ham', 0)
print(len(emails))
print(len(labels))

from nltk.corpus import names
from nltk.stem import WordNetLemmatizer
def is_letter_only(word):
    return word.isalpha()
all_names = set(names.words())
lemmatizer = WordNetLemmatizer()
def clean_text(docs):
    docs_cleaned = []
    for doc in docs:
        doc = doc.lower()
        doc_cleaned = ' '.join(lemmatizer.lemmatize(word) for word in doc.split() if is_letter_only(word))
        docs_cleaned.append(doc_cleaned)
    return docs_cleaned
emails_cleaned= clean_text(emails)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(emails_cleaned, labels, test_size=0.33, random_state=42)
print(len(X_train), len(Y_train))
print(len(X_test), len(Y_test))

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english", max_features=1000, max_df=0.5, min_df=2)

term_docs_train = cv.fit_transform(X_train)
term_docs_test = cv.transform(X_test)

1500
1500
5172
5172
3465 3465
1707 1707


Create multinomial NB and fit it

In [2]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(term_docs_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Predict probability

In [3]:
prediction_prob = clf.predict_proba(term_docs_test)
prediction_prob[0:10]

array([[1.00000000e+00, 3.96500362e-13],
       [1.00000000e+00, 2.15303766e-81],
       [6.59774100e-01, 3.40225900e-01],
       [1.00000000e+00, 2.28043493e-15],
       [1.00000000e+00, 1.77156705e-15],
       [5.53261316e-05, 9.99944674e-01],
       [0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 3.49697719e-28],
       [1.00000000e+00, 4.43498548e-14],
       [3.39263684e-01, 6.60736316e-01]])

Predict 1/0 value

In [4]:
prediction = clf.predict(term_docs_test)
prediction[:10]

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 1])

In [5]:
accuracy = clf.score(term_docs_test, Y_test)
print('The accuracy using MultinomialNB is: {0:.1f}%'.format(accuracy*100))

The accuracy using MultinomialNB is: 93.0%
